# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("/FileStore/tables/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [7]:
# Print the Schema of the DataFrame
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [8]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
 hduke@hotmail.com|4547 Archer Commo...| DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|
 pallen@yahoo.com|24645 Valerie Uni...| Bisque|33.000914755642675|11.330278057777512|37.110597442120856| 4.104543202376424| 487.54750486747207|
riverarebecca@gma...|1414 David Throug...| SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313| 3.120178782748092| 581.8523440352177|
mstephens@davidso...|14023 Rodriguez P...|MediumAquaMarine| 33.33067252364639|12.795188551078114| 37.53665330059473| 4.446308318351434| 599.4060920457634|
alvareznancy@luca...|645 Martha Park A...| FloralWhite|33.871037879341976|12.026925339755056| 34.47687762925054| 5.493507201364199| 637.102447915074|
katherine20@yahoo...|68388 Reyes Light...| DarkSlateBlue| 32.02159550138701|11.366348309710526| 36.68377615286961| 4.685017246570912| 521.5721747578274|
 awatkins@yahoo.com|Unit 6538 Box 898...| Aqua|32.739142938380326| 12.35195897300293| 37.37335885854755| 4.4342734348999375| 549.9041461052942|
vchurch@walter-ma...|860 Lee KeyWest D...| Salmon| 33.98777289568564|13.386235275676436|37.534497341555735| 3.2734335777477144| 570.2004089636196|
 bonnie69@lin.biz|PSC 2734, Box 525...| Brown|31.936548618448917|11.814128294972196| 37.14516822352819| 3.202806071553459| 427.1993848953282|
andrew06@peterson...|26104 Alexander G...| Tomato|33.992572774953736|13.338975447662113| 37.22580613162114| 2.482607770510596| 492.6060127179966|
ryanwerner@freema...|Unit 2413 Box 034...| Tomato| 33.87936082480498|11.584782999535266| 37.08792607098381| 3.71320920294043| 522.3374046069357|
 knelson@gmail.com|6705 Miller Orcha...| RoyalBlue|29.532428967057943|10.961298400154098| 37.42021557502538| 4.046423164299585| 408.6403510726275|
wrightpeter@yahoo...|05302 Dunlap Ferr...| Bisque| 33.19033404372265|12.959226091609382|36.144666700041924| 3.918541839158999| 573.4158673313865|
taylormason@gmail...|7773 Powell Sprin...| DarkBlue|32.387975853153876|13.148725692056516| 36.61995708279922| 2.494543646659249| 470.4527333009554|
 jstark@anderson.com|49558 Ramirez Roa...| Peru|30.737720372628182|12.636606052000127|36.213763093698624| 3.3578468423262944| 461.7807421962299|
 wjennings@gmail.com|6362 Wilson Mount...| PowderBlue| 32.12538689728784|11.733861690857394| 34.8940927514398| 3.1361327164897803| 457.84769594494855|
rebecca45@hale-ba...|8982 Burton RowWi...| OliveDrab|32.338899323067196|12.013194694014402| 38.38513659413844| 2.420806160901484| 407.70454754954415|
alejandro75@hotma...|64475 Andre Club ...| Cyan|32.187812045932155| 14.7153875441565| 38.24411459434352| 1.516575580831944| 452.3156754800354|
samuel46@love-wes...|544 Alexander Hei...| LightSeaGreen| 32.61785606282345|13.989592555825254|37.190503800397956| 4.064548550437977| 605.061038804892|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 20 rows

In [9]:
data.head(1)

Out[8]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [10]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

## Setting Up DataFrame for Machine Learning

In [12]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
# In this example, we catch only the numeric variables

# Yearly Amount Spent = variável dependente - variável que iremos prever

data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [14]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [15]:
output = assembler.transform(data)

In [16]:
output.printSchema()

output.select("features").show(5)

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

+--------------------+
 features|
+--------------------+
[34.4972677251122...|
[31.9262720263601...|
[33.0009147556426...|
[34.3055566297555...|
[33.3306725236463...|
+--------------------+
only showing top 5 rows

In [17]:
output.head(1)

Out[28]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [18]:
final_data = output.select("features",'Yearly Amount Spent')

final_data.show(2)

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
+--------------------+-------------------+
only showing top 2 rows

In [19]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 328|
 mean| 497.857456317286|
 stddev| 75.94312770408924|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [21]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 172|
 mean| 502.09170614666004|
 stddev| 85.53844674560919|
 min| 282.4712457199145|
 max| 725.5848140556806|
+-------+-------------------+

In [22]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [24]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} \n Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [26.172399766181723,38.92147690027186,0.15234765574506925,61.56643246917049] 
 Intercept: -1057.7415399958672

In [25]:
test_results = lrModel.evaluate(test_data)

In [26]:
# Interesting results....
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 11.992710990836486|
 -4.595425952135713|
 1.0646199270512966|
 10.959273374873987|
-12.460561234300883|
 -21.4573114546007|
 22.131148015096187|
 1.4557137563831475|
-3.8106818132778244|
 3.2615338633792703|
 -4.518355956924552|
 -5.05621116183795|
 -8.291865244142343|
 4.159190882858695|
 6.895710987657083|
-26.218572741721914|
-1.5733617429711444|
 -18.074849715119|
 -2.232787258700853|
 -9.122719710406443|
+-------------------+
only showing top 20 rows

In [27]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print('MAE: {}'.format(test_results.meanAbsoluteError))
print('Adj. R2: {}'.format(test_results.r2adj))

RMSE: 10.734284761183677
MSE: 115.22486933418008
MAE: 8.59010821894766
Adj. R2: 0.9837805740080232

In [28]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [29]:
unlabeled_data = test_data.select('features')

In [30]:
predictions = lrModel.transform(unlabeled_data)

In [31]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...| 396.647640081791|
[30.4925366965402...|287.06667167205023|
[30.5743636841713...|440.99979383101436|
[30.7377203726281...| 450.8214688213559|
[31.0662181616375...|461.39385444197524|
[31.1239743499119...|508.40436529436647|
[31.2834474760581...| 569.6499414105713|
[31.3895854806643...|408.61389730359974|
[31.4252268808548...| 534.5774004680397|
[31.4459724827577...| 481.6154310717493|
[31.5147378578019...|494.33084395338597|
[31.5257524169682...|449.02183797171983|
[31.5261978982398...|417.38639143648015|
[31.5316044825729...|432.35641484650387|
[31.6548096756927...|468.36771273989143|
[31.6739155032749...| 501.9436406516031|
[31.7216523605090...| 349.3502883748438|
[31.8164283341993...| 519.1973412187754|
[31.8186165667690...| 448.6514606288365|
[31.8279790554652...|449.12546725734796|
+--------------------+------------------+
only showing top 20 rows

Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!